# Input Functions

In [2]:
def AskForUserInput(df):
    fav_movie=input("Enter your Favorite Movie: ").lower()
    n=0
    movies=df["title"].unique()
    while fav_movie not in movies and n<5:
        print("The Movie ", fav_movie," does not exist in our database.")
        fav_movie = input("Please enter another favourite Movie: ").lower()
        n+=1
    return fav_movie

# KNN with Means class, and other functions

### Recommender System Class
(Inspired from https://www.kaggle.com/code/rangarajansaranathan/collaborative-filtering-based-recommender-system)
In [1]:

In [1]:
from collections import defaultdict

def get_top_n(predictions,df, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        #cross-relate other information from the fulldf
        movieName=df[df['movieId']==iid]['title'].unique()[0]
        movieYear=df[df['movieId']==iid]['year'].unique()[0]
        genres=df[df['movieId']==iid]['genres'].unique()[0]
        avgRat=df[df['movieId']==iid]['rating'].mean().round(2)
        ratedBy=len(df[df['movieId']==iid]['rating'])
        
        top_n[uid].append((iid, movieName, movieYear, genres, avgRat, ratedBy))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def rated_already(uid,df):
    df_out=df[df['userId']==uid]
    return df_out.title.unique()
    
class collab_filtering_Kmeans_Model():
    def __init__(self, model,trainset, testset):
        self.model = model
        self.trainset = trainset
        self.testset = testset

    def fit_and_predict(self,df):        
        print('**Fitting the train data...**')
        self.model.fit(self.trainset)       

        print('**Predicting the test data...**')
        pred_test = self.model.test(self.testset)        
        rmse = round(accuracy.rmse(pred_test), 3)
        print('**RMSE for the predicted result is ' + str(rmse) + '**')   
        
        #display(self.pred_test)
        top_n = get_top_n(pred_test,df)
      
        recommenddf = pd.DataFrame(columns=['userId', 'movieId', 'title', 'year', 'genres', 'average rating','number of ratings'])
        
        for item in top_n:
            subdf = pd.DataFrame(top_n[item], columns=['movieId','title',  'year', 'genres', 'average rating','number of ratings'])
            subdf['userId'] = item
            
            cols = subdf.columns.tolist()
            cols = cols[-1:] + cols[:-1]
            subdf = subdf[cols]        
            recommenddf = pd.concat([recommenddf, subdf], axis = 0)        
        return rmse, recommenddf
        
    def cross_validate(self,data):
        print('**Cross Validating the data...**')
        cv_result = cross_validate(self.model, data, n_jobs=-1,cv=5,verbose = True)
        
        return cv_result